In [6]:
import numpy as np
# load data in train_test_data file. X_train, X_test, y_train, y_test in .npy format
X_train = np.load('../train_test_data/X_train.npy')
X_test = np.load('../train_test_data/X_test.npy')
y_train = np.load('../train_test_data/y_train.npy')
y_test = np.load('../train_test_data/y_test.npy')

In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [8]:
model = keras.Sequential()
model.add(layers.LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(13, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 64)                19712     
                                                                 
 dense_2 (Dense)             (None, 128)               8320      
                                                                 
 dense_3 (Dense)             (None, 13)                1677      
                                                                 
Total params: 29709 (116.05 KB)
Trainable params: 29709 (116.05 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
from tensorflow import keras
from sklearn.model_selection import ParameterGrid

# Define a function to build the model with hyperparameters
def build_model(units, dense_units, dense_activation, learning_rate):
    model = keras.Sequential()
    model.add(layers.LSTM(units, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(layers.Dense(dense_units, activation=dense_activation))
    model.add(layers.Dense(13, activation='softmax'))
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Define hyperparameters to search over
hyperparameters = {
    'units': [32, 64, 128],
    'dense_units': [64, 128, 256],
    'dense_activation': ['relu', 'tanh', 'sigmoid'],
    'learning_rate': [0.01, 0.001, 0.0001]
}

# Perform a grid search over hyperparameters
best_accuracy = 0.0
best_hyperparameters = None

for params in ParameterGrid(hyperparameters):
    model = build_model(**params)
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
    _, accuracy = model.evaluate(X_test, y_test)
    
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_hyperparameters = params

print("Best hyperparameters:", best_hyperparameters)


Epoch 1/10
151/151 [==============================] - 9s 39ms/step - loss: 0.7126 - accuracy: 0.7307 - val_loss: 0.4395 - val_accuracy: 0.8227
Epoch 2/10
151/151 [==============================] - 5s 31ms/step - loss: 0.2910 - accuracy: 0.8930 - val_loss: 0.2025 - val_accuracy: 0.9330
Epoch 3/10
151/151 [==============================] - 5s 30ms/step - loss: 0.1347 - accuracy: 0.9540 - val_loss: 0.1808 - val_accuracy: 0.9461
Epoch 4/10
151/151 [==============================] - 5s 30ms/step - loss: 0.1124 - accuracy: 0.9629 - val_loss: 0.1444 - val_accuracy: 0.9509
Epoch 5/10
151/151 [==============================] - 4s 29ms/step - loss: 0.1140 - accuracy: 0.9679 - val_loss: 0.1537 - val_accuracy: 0.9563
Epoch 6/10
151/151 [==============================] - 5s 34ms/step - loss: 0.1008 - accuracy: 0.9706 - val_loss: 0.1895 - val_accuracy: 0.9461
Epoch 7/10
151/151 [==============================] - 5s 34ms/step - loss: 0.0595 - accuracy: 0.9835 - val_loss: 0.1365 - val_accuracy: 0.9582

In [9]:
callbacks = [keras.callbacks.ModelCheckpoint("../models/LSTM_simple.h5", save_best_only=True, monitor="val_loss"),
             keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1)]
 
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["sparse_categorical_accuracy"],)
 
model_history = model.fit(X_train,y_train, epochs= 10, validation_data=(X_test,y_test), callbacks=callbacks)

Epoch 1/10
151/151 [==============================] - 7s 29ms/step - loss: 0.9236 - sparse_categorical_accuracy: 0.7168 - val_loss: 0.3633 - val_sparse_categorical_accuracy: 0.9058
Epoch 2/10
151/151 [==============================] - 4s 24ms/step - loss: 0.2501 - sparse_categorical_accuracy: 0.9269 - val_loss: 0.2304 - val_sparse_categorical_accuracy: 0.9276
Epoch 3/10
151/151 [==============================] - 4s 24ms/step - loss: 0.1214 - sparse_categorical_accuracy: 0.9615 - val_loss: 0.1377 - val_sparse_categorical_accuracy: 0.9475
Epoch 4/10
151/151 [==============================] - 4s 24ms/step - loss: 0.1066 - sparse_categorical_accuracy: 0.9638 - val_loss: 0.1351 - val_sparse_categorical_accuracy: 0.9645
Epoch 5/10
151/151 [==============================] - 4s 24ms/step - loss: 0.0680 - sparse_categorical_accuracy: 0.9790 - val_loss: 0.1064 - val_sparse_categorical_accuracy: 0.9660
Epoch 6/10
151/151 [==============================] - 4s 24ms/step - loss: 0.0362 - sparse_cate

In [10]:
model = keras.models.load_model('../models/LSTM_simple.h5')

train_loss, train_acc = model.evaluate(X_train,y_train)
test_loss, test_acc = model.evaluate(X_test,y_test)

print("Train accuracy", round(train_acc*100, 2),'%')
print("Train loss", train_loss)
print("Test accuracy", round(test_acc*100, 2),'%')
print("Test loss", test_loss)

ValueError: not enough values to unpack (expected 3, got 2)